# Interval methods for linear equation 

In [143]:
!pip install -q pyinterval

In [144]:
from interval import interval
from interval import imath
import pandas as pd
import numpy as np
import math

In [145]:
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_midpoint(interv):
    return (interv[0].sup + interv[0].inf) / 2

def interval_to_str(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

def to_table(interv, interv_ext, width, middle, root, tbl):
    tbl.loc[len(tbl)] = [interv, interv_ext, width, middle,root]

In [146]:
func = lambda x: (x-1)*(x+1)*(x+2)*imath.sin(x)
dfunc = lambda x: (3*x**2+4*x-1)*imath.sin(x)+(x**3+2*x**2-x-2)*imath.cos(x)
e = 0.01
a = 0.0
b = 2.3

## Interval division 

In [147]:
def interval_division(f, start, end, e, tbl):
    x = interval[start, end]
    width = calculate_width(x)
    mid = calculate_midpoint(x)
    Fx = f(x)
    root_exists = 0 in Fx and width < e
    
    if 0 not in Fx or root_exists :
        to_table(interval_to_str(x), interval_to_str(Fx), width, mid, 'Root' if root_exists else '', tbl)
        return   
        
    tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, mid, '']
    interval_division(f, start, mid, e, tbl)
    interval_division(f, mid, end, e, tbl)

In [148]:
interval_division_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Middle point', 'Root'])
interval_division(func, a, b, e, interval_division_result)
interval_division_result

,Interval,Interval extension,Width,Middle point,Root
0,"[0.0000000, 2.3000000]","[-14.1900000, 18.4470000]",2.300000,1.150000,
1,"[0.0000000, 1.1500000]","[-6.1816938, 0.9272541]",1.150000,0.575000,
2,"[0.0000000, 0.5750000]","[-2.2055900, -0.0000000]",0.575000,0.287500,
3,"[0.0000000, 0.2875000]","[-0.8351159, -0.0000000]",0.287500,0.143750,
4,"[0.0000000, 0.1437500]","[-0.3512500, -0.0000000]",0.143750,0.071875,
5,"[0.0000000, 0.0718750]","[-0.1594820, -0.0000000]",0.071875,0.035937,
6,"[0.0000000, 0.0359375]","[-0.0757796, -0.0000000]",0.035937,0.017969,
7,"[0.0000000, 0.0179687]","[-0.0369099, -0.0000000]",0.017969,0.008984,
8,"[0.0000000, 0.0089844]","[-0.0182114, -0.0000000]",0.008984,0.004492,Root
9,"[0.0089844, 0.0179687]","[-0.0365783, -0.0178842]",0.008984,0.013477,


## Moore's method modification 

In [149]:
def moore(f, df, start, end, e, tbl):
    x = interval([start, end])
    width = calculate_width(x)
    Fx = f(x)
    root_exists = 0 in Fx and width < e

    if 0 not in Fx or root_exists :
        to_table(interval_to_str(x), interval_to_str(Fx), width, calculate_midpoint(x),
                     'Root' if root_exists else '', tbl)
        return 
        
    x_middle = calculate_midpoint(x)
    dFx = df(x)
    if 0 in dFx:
        to_table(interval_to_str(x), interval_to_str(Fx), width, x_middle, '', tbl)
        moore(f, df, start, x_middle, e, tbl)
        moore(f, df, x_middle, end, e, tbl)
        return
        
    f_middle = f(x_middle)
    N = x_middle - f_middle / dFx
    x_next = N & x
    if not x_next:
        to_table(interval_to_str(x), interval_to_str(Fx), width, x_middle, '', tbl)
        return

    to_table(interval_to_str(x), interval_to_str(Fx), width, x_middle, '', tbl)
    moore(f, df, x_next[0].inf, x_next[0].sup, e, tbl)

In [150]:
moore_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Middle point', 'Root'])
moore(func, dfunc, a, b, e, moore_result)
moore_result

,Interval,Interval extension,Width,Middle point,Root
0,"[0.0000000, 2.3000000]","[-14.1900000, 18.4470000]",2.300000,1.150000,
1,"[0.0000000, 1.1500000]","[-6.1816938, 0.9272541]",1.150000,0.575000,
2,"[0.0000000, 0.5750000]","[-2.2055900, -0.0000000]",0.575000,0.287500,
3,"[0.0000000, 0.2875000]","[-0.8351159, -0.0000000]",0.287500,0.143750,
4,"[0.0000000, 0.0267717]","[-0.0557060, -0.0000000]",0.026772,0.013386,
5,"[0.0000000, 0.0002645]","[-0.0005292, -0.0000000]",0.000264,0.000132,Root
6,"[0.2875000, 0.5750000]","[-1.5714829, -0.3549243]",0.287500,0.431250,
7,"[0.5750000, 1.1500000]","[-2.6272199, 0.9272541]",0.575000,0.862500,
8,"[0.5750000, 0.8625000]","[-1.7208462, -0.3032686]",0.287500,0.718750,
9,"[0.8625000, 1.1500000]","[-0.8499829, 0.9272541]",0.287500,1.006250,


## Hansen's method modification 

In [151]:
def hansen(f, df, start, end, e, tbl):
    x = interval([start, end])
    width = calculate_width(x)
    Fx = f(x)
    root_exists = 0 in Fx and width < e

    if 0 not in Fx or root_exists :
        to_table(interval_to_str(x), interval_to_str(Fx), width, calculate_midpoint(x),
                     'Root' if root_exists else '', tbl)
        return 
        
    x_middle = calculate_midpoint(x)
    f_middle = f(x_middle)
    dFx = df(x)
    if f_middle == 0.0:
        to_table(interval_to_str(x), interval_to_str(Fx), width, x_middle, '', tbl)
        hansen(f, df, start, x_middle, tbl)
        hansen(f, df, x_middle, end, tbl)
        return

    U = x_middle - f_middle / dFx
    x_next = U & x
    if not x_next:
        to_table(interval_to_str(x), interval_to_str(Fx), width, x_middle, '', tbl)
        return

    to_table(interval_to_str(x), interval_to_str(Fx), width, x_middle, '', tbl)
    [hansen(f, df, x.inf, x.sup, e, tbl) for x in x_next]

In [152]:
hansen_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Middle point', 'Root'])
hansen(func, dfunc, a, b, e, hansen_result)
hansen_result

,Interval,Interval extension,Width,Middle point,Root
0,"[0.0000000, 2.3000000]","[-14.1900000, 18.4470000]",2.300000,1.150000,
1,"[0.0000000, 1.1293102]","[-6.0243822, 0.7790142]",1.129310,0.564655,
2,"[0.0000000, 0.3328828]","[-1.0160746, -0.0000000]",0.332883,0.166441,
3,"[0.0000000, 0.0352288]","[-0.0742092, -0.0000000]",0.035229,0.017614,
4,"[0.0000000, 0.0004557]","[-0.0009121, -0.0000000]",0.000456,0.000228,Root
5,"[0.6729778, 1.1293102]","[-1.9701070, 0.7790142]",0.456332,0.901144,
6,"[0.9568807, 1.1293102]","[-0.2597672, 0.7790142]",0.172430,1.043095,
7,"[0.9894281, 1.0110990]","[-0.0542509, 0.0569558]",0.021671,1.000264,
8,"[0.9999908, 1.0000089]","[-0.0000466, 0.0000450]",0.000018,1.000000,Root
9,"[1.2125541, 2.3000000]","[1.1266294, 18.4470000]",1.087446,1.756277,


## Krawczyk's method modification 

In [153]:
def krawczyk(f, df, start, end, e, tbl):
    x = interval([start, end])
    width = calculate_width(x)
    Fx = f(x)
    root_exists = 0 in Fx and width < e

    if 0 not in Fx or root_exists :
        to_table(interval_to_str(x), interval_to_str(Fx), width, calculate_midpoint(x),
                     'Root' if root_exists else '', tbl)
        return

    x_middle = calculate_midpoint(x)
    dFx = df(x)
    if 0 in dFx:
        to_table(interval_to_str(x), interval_to_str(Fx), width, x_middle, '', tbl)
        krawczyk(f, df, start, x_middle, e, tbl)
        krawczyk(f, df, x_middle, end, e, tbl)
        return

    dFx_middle = df(x_middle)
    K = x_middle - f(x_middle) / dFx_middle + (1 - dFx / dFx_middle)*(x - x_middle)
    x_next = K & x
    if not x_next:
        to_table(interval_to_str(x), interval_to_str(Fx), width, x_middle, '', tbl)
        return

    to_table(interval_to_str(x), interval_to_str(Fx), width, x_middle, '', tbl)
    krawczyk(f, df, x_next[0].inf, x_next[0].sup, e, tbl)

In [154]:
krawczyk_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Middle point', 'Root'])
krawczyk(func, dfunc, a, b, e, krawczyk_result)
krawczyk_result

,Interval,Interval extension,Width,Middle point,Root
0,"[0.0000000, 2.3000000]","[-14.1900000, 18.4470000]",2.300000,1.150000,
1,"[0.0000000, 1.1500000]","[-6.1816938, 0.9272541]",1.150000,0.575000,
2,"[0.0000000, 0.5750000]","[-2.2055900, -0.0000000]",0.575000,0.287500,
3,"[0.0000000, 0.2875000]","[-0.8351159, -0.0000000]",0.287500,0.143750,
4,"[0.0000000, 0.0366916]","[-0.0774539, -0.0000000]",0.036692,0.018346,
5,"[0.0000000, 0.0005034]","[-0.0010076, -0.0000000]",0.000503,0.000252,Root
6,"[0.2875000, 0.5750000]","[-1.5714829, -0.3549243]",0.287500,0.431250,
7,"[0.5750000, 1.1500000]","[-2.6272199, 0.9272541]",0.575000,0.862500,
8,"[0.5750000, 0.8625000]","[-1.7208462, -0.3032686]",0.287500,0.718750,
9,"[0.8625000, 1.1500000]","[-0.8499829, 0.9272541]",0.287500,1.006250,
